In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('data/train_set.csv')
test = pd.read_csv('data/test_set.csv')
data = pd.concat([train,test])

In [3]:
feature = train.columns.tolist()
feature.remove('ID')
feature.remove('y')
object_columns = train.columns[train.dtypes == 'object'].tolist()
num_columns = list(set(feature) - set(object_columns))
print (object_columns)
print (num_columns)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
['age', 'balance', 'pdays', 'campaign', 'duration', 'day', 'previous']


In [4]:
train[num_columns].describe()

,age,balance,pdays,campaign,duration,day,previous
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,40.935379,1357.555082,40.248766,2.772050,257.732393,15.835289,0.591737
std,10.634289,2999.822811,100.213541,3.136097,256.975151,8.319480,2.568313
min,18.000000,-8019.000000,-1.000000,1.000000,0.000000,1.000000,0.000000
25%,33.000000,73.000000,-1.000000,1.000000,103.000000,8.000000,0.000000
50%,39.000000,448.000000,-1.000000,2.000000,181.000000,16.000000,0.000000
75%,48.000000,1435.000000,-1.000000,3.000000,317.000000,21.000000,0.000000
max,95.000000,102127.000000,854.000000,55.000000,3881.000000,31.000000,275.000000


In [5]:
for col in object_columns:
    data = pd.concat([data, pd.get_dummies(data[col], prefix=col+'_')], axis=1)
    data.drop(col, axis=1, inplace=True)

In [6]:
X_train = data[data['y'].notnull()]
X_test = data[data['y'].isnull()]

y_train = X_train['y']
X_train.drop(['ID', 'y'], axis=1, inplace=True)
result = pd.DataFrame({'ID': X_test['ID']})
X_test.drop(['ID', 'y'], axis=1, inplace=True)

In [7]:
scaler = StandardScaler()
scaler.fit(X_train[num_columns])
X_train[num_columns] = scaler.transform(X_train[num_columns])
X_test[num_columns] = scaler.transform(X_test[num_columns])

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train, y_train) # 将数据保存到LightGBM二进制文件将使加载更快
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)  # 创建验证数据
 
# 将参数写成字典下形式
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'auc'},
    'learning_rate': 0.01,
    'is_unbalance': True,
    'random_state': 0,
    'verbose': 0
}
 
print('Start training...')
clf = lgb.train(params,
                lgb_train,
                valid_sets = lgb_val,
                num_boost_round = 10000,
                verbose_eval = 100,
                early_stopping_rounds = 100)
 
print('Start predicting...')
# 预测数据集
y_pred = clf.predict(X_val, num_iteration=clf.best_iteration) #如果在训练期间启用了早期停止，可以通过best_iteration方式从最佳迭代中获得预测
# 评估模型
print(roc_auc_score(y_val, y_pred))

Start training...
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.916089
[200]	valid_0's auc: 0.92064
[300]	valid_0's auc: 0.92451
[400]	valid_0's auc: 0.926806
[500]	valid_0's auc: 0.927627
[600]	valid_0's auc: 0.928324
[700]	valid_0's auc: 0.928518
Early stopping, best iteration is:
[652]	valid_0's auc: 0.928625
Start predicting...
0.9286252235563214


In [9]:
y_test = clf.predict(X_test)
result['pred'] = y_test
print (result.head())
result.to_csv('data/submission.csv', index=False)

      ID      pred
0  25318  0.161860
1  25319  0.024491
2  25320  0.006308
3  25321  0.903426
4  25322  0.090160


In [10]:
# param = {
#     'task': 'train',
#     'boosting_type': 'gbdt',  # 设置提升类型
#     'objective': 'binary', # 目标函数
#     'metric': {'auc'},  # 评估函数
#     'learning_rate': 0.01,
#     'is_unbalance': True,
#     'verbose': 0 # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
# }
# folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
# oof = np.zeros(len(y_train))
# result['pred'] = 0
# feature_importance_df = pd.DataFrame()

# for fold_, (tra_idx, val_idx) in enumerate(folds.split(X_train, y_train.values)):
#     print("fold {}".format(fold_))
#     lgb_train = lgb.Dataset(X_train.iloc[tra_idx], label=y_train.iloc[tra_idx])#, categorical_feature=categorical_feats)
#     lgb_val = lgb.Dataset(X_train.iloc[val_idx], label=y_train.iloc[val_idx])#, categorical_feature=categorical_feats)
#     clf = lgb.train(param, 
#                     lgb_train, 
#                     num_boost_round = 10000,
#                     valid_sets = [lgb_train, lgb_val], 
#                     verbose_eval=100, 
#                     early_stopping_rounds = 200)
#     oof[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
#     fold_importance_df = pd.DataFrame()
#     fold_importance_df["Feature"] = X_train.columns.tolist()
#     fold_importance_df["importance"] = clf.feature_importance()
#     fold_importance_df["fold"] = fold_
#     feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
#     result['pred'] += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits

In [11]:
# from sklearn.metrics import roc_auc_score
# roc_auc_score(y_train, oof)

In [12]:
# print (result.head())
# result.to_csv('data/submission.csv', index=False)

In [13]:
# feature_importance_sort = feature_importance_df[["Feature", "importance"]].groupby("Feature").mean()\
#                             .sort_values(by="importance", ascending=False)
# feature_importance_sort[:20]